In [4]:
import streamlit as st
import pandas as pd
import numpy as np
import pydeck as pdk
import plotly.express as px


In [5]:
DATE_TIME = "ACCIDENT DATE_ACCIDENT TIME"
DATA_URL = (r"nypd.csv")

In [10]:
da = pd.read_csv('nyc.csv',encoding='latin1')
da.columns

ParserError: Error tokenizing data. C error: Expected 1 fields in line 12, saw 2


In [ ]:
da.shape

(1612178, 29)

In [ ]:
da.head()

,ACCIDENT DATE,ACCIDENT TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2019-08-05T00:00:00.000,16:30,QUEENS,11434.0,40.676052,-73.790184,"{'type': 'Point', 'coordinates': [-73.790184, ...",NaN,NaN,150-08 123 AVENUE,...,Unspecified,NaN,NaN,NaN,4184637,Sedan,Pick-up Truck,NaN,NaN,NaN
1,2019-08-27T00:00:00.000,16:02,BROOKLYN,11225.0,40.657780,-73.951096,"{'type': 'Point', 'coordinates': [-73.951096, ...",NaN,NaN,288 HAWTHORNE STREET,...,Unspecified,NaN,NaN,NaN,4195773,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
2,2019-08-15T00:00:00.000,17:57,MANHATTAN,10002.0,40.718143,-73.993835,"{'type': 'Point', 'coordinates': [-73.993835, ...",CHRYSTIE STREET,GRAND STREET,NaN,...,NaN,NaN,NaN,NaN,4202457,Sedan,NaN,NaN,NaN,NaN
3,2019-08-30T00:00:00.000,21:53,BRONX,10460.0,40.840534,-73.866610,"{'type': 'Point', 'coordinates': [-73.86661, 4...",NaN,NaN,1837 EAST TREMONT AVENUE,...,Unspecified,NaN,NaN,NaN,4198749,Taxi,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
4,2019-08-06T00:00:00.000,9:45,MANHATTAN,10016.0,40.745440,-73.975400,"{'type': 'Point', 'coordinates': [-73.9754, 40...",EAST 35 STREET,2 AVENUE,NaN,...,Driver Inattention/Distraction,NaN,NaN,NaN,4183798,Station Wagon/Sport Utility Vehicle,Bike,NaN,NaN,NaN


In [ ]:
@st.cache_data(persist=True)
def load_data(nrows):
    data = pd.read_csv(DATA_URL, nrows=nrows, parse_dates=[['ACCIDENT DATE', 'ACCIDENT TIME']])
    data.dropna(subset=['LATITUDE', 'LONGITUDE'], inplace=True)
    lowercase = lambda x: str(x).lower()
    data.rename(lowercase, axis="columns", inplace=True)
    data.rename(columns={"number of persons injured": "number_of_persons_injured"}, inplace=True)
    data.rename(columns={"number of pedestrians injured": "number_of_pedestrians_injured"}, inplace=True)
    data.rename(columns={"number of cyclist injured": "number_of_cyclists_injured"}, inplace=True)
    data.rename(columns={"number of motorist injured": "number_of_motorists_injured"}, inplace=True)
    data.rename(columns={"on street name": "on_street_name"}, inplace=True)
    #data.rename(columns={"crash_date_crash_time": "crash date/ crash time"}, inplace=True)
    #data = data[['date/time', 'latitude', 'longitude']]
    return data

data = load_data(2200)
data[['latitude','longitude']].to_csv('lat_long.csv', index=False)

2023-09-30 19:38:26.764 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-09-30 19:38:26.834 
  command:

    streamlit run /Users/baba/Library/Python/3.11/lib/python/site-packages/ipykernel_launcher.py [ARGUMENTS]
2023-09-30 19:38:26.835 No runtime found, using MemoryCacheStorageManager
/var/folders/g4/266dj37n1f7_4d9mw_50bmx80000gn/T/ipykernel_36371/3287075304.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data = pd.read_csv(DATA_URL, nrows=nrows, parse_dates=[['ACCIDENT DATE', 'ACCIDENT TIME']])


In [ ]:
data.head()

,accident date_accident time,borough,zip code,latitude,longitude,location,on_street_name,cross street name,off street name,number_of_persons_injured,...,contributing factor vehicle 2,contributing factor vehicle 3,contributing factor vehicle 4,contributing factor vehicle 5,collision_id,vehicle type code 1,vehicle type code 2,vehicle type code 3,vehicle type code 4,vehicle type code 5
0,2019-08-05 16:30:00,QUEENS,11434.0,40.676052,-73.790184,"{'type': 'Point', 'coordinates': [-73.790184, ...",NaN,NaN,150-08 123 AVENUE,0,...,Unspecified,NaN,NaN,NaN,4184637,Sedan,Pick-up Truck,NaN,NaN,NaN
1,2019-08-27 16:02:00,BROOKLYN,11225.0,40.657780,-73.951096,"{'type': 'Point', 'coordinates': [-73.951096, ...",NaN,NaN,288 HAWTHORNE STREET,0,...,Unspecified,NaN,NaN,NaN,4195773,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
2,2019-08-15 17:57:00,MANHATTAN,10002.0,40.718143,-73.993835,"{'type': 'Point', 'coordinates': [-73.993835, ...",CHRYSTIE STREET,GRAND STREET,NaN,1,...,NaN,NaN,NaN,NaN,4202457,Sedan,NaN,NaN,NaN,NaN
3,2019-08-30 21:53:00,BRONX,10460.0,40.840534,-73.866610,"{'type': 'Point', 'coordinates': [-73.86661, 4...",NaN,NaN,1837 EAST TREMONT AVENUE,0,...,Unspecified,NaN,NaN,NaN,4198749,Taxi,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
4,2019-08-06 09:45:00,MANHATTAN,10016.0,40.745440,-73.975400,"{'type': 'Point', 'coordinates': [-73.9754, 40...",EAST 35 STREET,2 AVENUE,NaN,1,...,Driver Inattention/Distraction,NaN,NaN,NaN,4183798,Station Wagon/Sport Utility Vehicle,Bike,NaN,NaN,NaN


In [ ]:
data.columns

Index(['accident date_accident time', 'borough', 'zip code', 'latitude',
       'longitude', 'location', 'on_street_name', 'cross street name',
       'off street name', 'number_of_persons_injured',
       'number of persons killed', 'number_of_pedestrians_injured',
       'number of pedestrians killed', 'number_of_cyclists_injured',
       'number of cyclist killed', 'number_of_motorists_injured',
       'number of motorist killed', 'contributing factor vehicle 1',
       'contributing factor vehicle 2', 'contributing factor vehicle 3',
       'contributing factor vehicle 4', 'contributing factor vehicle 5',
       'collision_id', 'vehicle type code 1', 'vehicle type code 2',
       'vehicle type code 3', 'vehicle type code 4', 'vehicle type code 5'],
      dtype='object')

In [ ]:
#Slider to filter based on no of injured people
st.header("Where are the most people injured in NYC?")
injured_people = st.slider("Number of persons injured in vehicle collisions", 0, 19)
st.map(data.query("number_of_persons_injured >= @injured_people")[["latitude", "longitude"]].dropna(how="any"))

#slider to filter based on time
st.header("How many collisions occur during a given time of day?")
hour = st.slider("Hour to look at", 0, 23)
original_data = data
data = data[data['accident date_accident time'].dt.hour == hour]
st.markdown("Vehicle collisions between %i:00 and %i:00" % (hour, (hour + 1) % 24))

2023-09-30 19:38:27.196 Session state does not function when running a script without `streamlit run`


DeltaGenerator()

In [ ]:
#Showing 3D Map and chart
midpoint = (np.average(data["latitude"]), np.average(data["longitude"]))
st.write(pdk.Deck(
    map_style="mapbox://styles/mapbox/light-v9",
    initial_view_state={
        "latitude": midpoint[0],
        "longitude": midpoint[1],
        "zoom": 11,
        "pitch": 50,
    },
    layers=[
        pdk.Layer(
        "HexagonLayer",
        data=data[['accident date_accident time', 'latitude', 'longitude']],
        get_position=["longitude", "latitude"],
        auto_highlight=True,
        radius=100,
        extruded=True,
        pickable=True,
        elevation_scale=4,
        elevation_range=[0, 1000],
        ),
    ],
))

In [ ]:
#Showing Histogram based on time
st.subheader("Breakdown by minute between %i:00 and %i:00" % (hour, (hour + 1) % 24))
filtered = data[
    (data['accident date_accident time'].dt.hour >= hour) & (data['accident date_accident time'].dt.hour < (hour + 1))
]
hist = np.histogram(filtered['accident date_accident time'].dt.minute, bins=60, range=(0, 60))[0]
chart_data = pd.DataFrame({"minute": range(60), "crashes": hist})

fig = px.bar(chart_data, x='minute', y='crashes', hover_data=['minute', 'crashes'], height=400)
st.write(fig)

In [ ]:
#Slider to filter based on no of injured pedestrians
st.header("Where are the most pedestrians injured in NYC?")
injured_pedestrians = st.slider("Number of pedestrians injured", 0, 19)
st.map(data.query("number_of_pedestrians_injured >= @injured_pedestrians")[["latitude", "longitude"]].dropna(how="any"))

DeltaGenerator()

In [ ]:
#Slider to filter based on no of injured cyclists
st.header("Where are the most cyclists injured in NYC?")
injured_cyclists = st.slider("Number of cyclists injured", 0, 19)
st.map(data.query("number_of_cyclists_injured >= @injured_cyclists")[["latitude", "longitude"]].dropna(how="any"))

DeltaGenerator()

In [ ]:
#Slider to filter based on no of injured motorists
st.header("Where are the most motorists injured in NYC?")
injured_motorists = st.slider("Number of motorists injured", 0, 19)
st.map(data.query("number_of_motorists_injured >= @injured_motorists")[["latitude", "longitude"]].dropna(how="any"))

DeltaGenerator()

In [ ]:
original_data.columns

Index(['accident date_accident time', 'borough', 'zip code', 'latitude',
       'longitude', 'location', 'on_street_name', 'cross street name',
       'off street name', 'number_of_persons_injured',
       'number of persons killed', 'number_of_pedestrians_injured',
       'number of pedestrians killed', 'number_of_cyclists_injured',
       'number of cyclist killed', 'number_of_motorists_injured',
       'number of motorist killed', 'contributing factor vehicle 1',
       'contributing factor vehicle 2', 'contributing factor vehicle 3',
       'contributing factor vehicle 4', 'contributing factor vehicle 5',
       'collision_id', 'vehicle type code 1', 'vehicle type code 2',
       'vehicle type code 3', 'vehicle type code 4', 'vehicle type code 5'],
      dtype='object')

In [ ]:
st.write(original_data.columns)


In [ ]:
st.write(original_data.head())


In [ ]:
#Showing Top 5 Dangerous streets by filtering
# Sidebar with user input
st.sidebar.header("Select Affected Class")
select = st.sidebar.selectbox("Affected class", ["Pedestrians", "Cyclists", "Motorists"])

# Filter and display top 5 dangerous streets
st.title("Top 5 Dangerous Streets by Affected Class")
if select == "Pedestrians":
    column_name = "number_of_pedestrians_injured"
elif select == "Cyclists":
    column_name = "number_of_cyclists_injured"
else:
    column_name = "number_of_motorists_injured"

# Filter the data based on the selected class and injury count
filtered_data = original_data[original_data[column_name] >= 1]

# Sort the data by the injury count in descending order and get the top 5
sorted_data = filtered_data.sort_values(by=[column_name], ascending=False).head(5)

# Display the results in a table
st.write(sorted_data[["on_street_name", column_name]])

In [ ]:
#Checkbox to show raw table data
if st.checkbox("Show raw data", False):
    st.subheader("Raw data by minute between %i:00 and %i:00" % (hour, (hour + 1) % 24))
    st.write(data)